# 🤖 Agentic Data Analysis with LangChain and Gradio

This notebook documents the setup, code, and execution of a web application that uses a LangChain agent to perform data analysis on user-uploaded CSV files. The interface is built with Gradio, and the project environment is managed by `uv`.

## Step 1: Project Setup with `uv`

We use `uv`, a fast Python package manager from Astral, to handle our project's virtual environment and dependencies. This ensures a clean, reproducible setup.

### 1.1 - Install `uv`

If you don't have `uv` installed, run the following command in your terminal.

In [14]:
# On macOS / Linux
!curl -LsSf https://astral.sh/uv/install.sh | sh

# On Windows (in PowerShell)
# powershell -ExecutionPolicy ByPass -c "irm https://astral.sh/uv/install.ps1 | iex"

'sh' is not recognized as an internal or external command,
operable program or batch file.


### 1.2 - Initialize Project and Install Dependencies

Create a project directory, initialize it, and add the required packages.

In [15]:
!mkdir agentic-data-app
!cd agentic-data-app
!uv init
!uv venv
!uv add langchain langchain-openai pandas matplotlib gradio ipykernel pip langchain-experimental

error: Project is already initialized in `c:\Users\miqba\projects\Analyst_Agent_with_Langchain` (`pyproject.toml` file exists)
Using CPython 3.12.3 interpreter at: C:\Users\miqba\anaconda3\python.exe
Creating virtual environment at: .venv
uv::venv::creation

  × Failed to create virtualenv
  ╰─▶ failed to remove directory
      `c:\Users\miqba\projects\Analyst_Agent_with_Langchain\.venv`: Access is
      denied. (os error 5)


^C


Resolved 130 packages in 4ms
Installed 122 packages in 8.56s
 + aiofiles==24.1.0
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.12.15
 + aiosignal==1.4.0
 + annotated-types==0.7.0
 + anyio==4.10.0
 + asttokens==3.0.0
 + attrs==25.3.0
 + brotli==1.1.0
 + certifi==2025.8.3
 + charset-normalizer==3.4.2
 + click==8.2.1
 + colorama==0.4.6
 + comm==0.2.3
 + contourpy==1.3.3
 + cycler==0.12.1
 + dataclasses-json==0.6.7
 + debugpy==1.8.15
 + decorator==5.2.1
 + distro==1.9.0
 + dotenv==0.9.9
 + executing==2.2.0
 + fastapi==0.116.1
 + ffmpy==0.6.1
 + filelock==3.18.0
 + fonttools==4.59.0
 + frozenlist==1.7.0
 + fsspec==2025.7.0
 + gradio==5.41.0
 + gradio-client==1.11.0
 + greenlet==3.2.3
 + groovy==0.1.2
 + h11==0.16.0
 + httpcore==1.0.9
 + httpx==0.28.1
 + httpx-sse==0.4.1
 + huggingface-hub==0.34.3
 + idna==3.10
 + ipykernel==6.30.1
 + ipython==9.4.0
 + ipython-pygments-lexers==1.1.1
 + jedi==0.19.2
 + jinja2==3.1.6
 + jiter==0.10.0
 + jsonpatch==1.33
 + jsonpointer==3.0.0
 + jupyter-client==8.6.3

## Step 2: The Application Code (`app.py`)

### 2.1 - Imports and Configuration

First, we import all the necessary libraries. We need `gradio` for the UI, `pandas` for data handling, `os` for file operations, and `langchain` components to build and run our agent. We also define a constant for the filename of our plots to keep the code clean.

In [ ]:
import gradio as gr
import pandas as pd
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI  # Changed from OpenAI to ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# --- Load Environment Variables ---
# This line reads the .env file and loads the variables into the environment
load_dotenv()
# --- Configuration ---
# Define a constant for the plot filename to ensure consistency.
PLOT_FILENAME = "temp_analysis_plot.png"

### 2.2 - The Agent Logic Function

Next, we define the main function, `data_analyst_agent`. This function will be called by Gradio every time the user clicks the "Run Analysis" button. It takes the uploaded file and the user's text prompt as input.

The first part of the function handles input validation and cleans up any old plot files.

In [ ]:
def data_analyst_agent(file_obj, user_prompt):
    """
    This is the core function that orchestrates the agent's work.
    """
    # 1. Input Validation
    if file_obj is None:
        return "Error: Please upload a CSV file first.", None
    if not user_prompt:
        return "Error: Please enter a question or instruction.", None

    # 2. Cleanup: Remove any old plot file to prevent showing stale results.
    if os.path.exists(PLOT_FILENAME):
        os.remove(PLOT_FILENAME)

### 2.3 - Create and Execute the Agent

This is the core of the function. We create an instance of the `create_pandas_dataframe_agent`, which is specifically designed to work with data in a pandas DataFrame.

We then create a detailed prompt that includes the user's original question plus specific instructions for the agent, telling it to save any plots it makes. This makes the agent much more reliable. Finally, we `invoke` the agent to run the analysis and handle any errors that might occur.

In [ ]:
try:
    # Load Data: The file_obj from Gradio has a .name attribute
    # which holds the temporary path to the uploaded file.
    df = pd.read_csv(file_obj.name)

    # Initialize LLM: Use a chat model that supports tool calling.
    llm = ChatOpenAI(
        api_key=api_key, 
        temperature=0, 
        model="gpt-3.5-turbo" # Specify a tool-calling model
    )

    # Create Agent: This is the heart of the operation.
    # `create_pandas_dataframe_agent` equips the LLM with tools
    # to execute pandas operations on the DataFrame.
    agent_executor = create_pandas_dataframe_agent(
        llm,
        df,
        agent_type="openai-tools",
        verbose=True,  # Set to True to see the agent's thought process in the terminal
        allow_dangerous_code=True # Opt-in to allow the agent to execute Python code
    )

    # Craft a Detailed Prompt: We augment the user's prompt to give the
    # agent explicit instructions on how to handle visualizations.
    # This makes the agent much more reliable.
    full_prompt = f"""
    User Question: {user_prompt}

    Instructions for the agent:
    - First, analyze the provided data to answer the user's question.
    - If you generate a plot or visualization, you MUST save it as a file named '{PLOT_FILENAME}'.
    - In your final answer, you must explicitly describe the visualization you created (e.g., "I have created a bar chart that shows the total sales for each product category.").
    - Also, mention that the plot has been saved.
    - Use Markdown formatting for all text in your output. This includes headings, bullet points, code blocks (if any), and emphasis for clarity.
    """

    # Run the Agent: Invoke the agent with the detailed prompt.
    response = agent_executor.invoke({"input": full_prompt})
    
    # Extract the text output from the agent's response.
    text_output = response.get('output', "I couldn't generate a text response. Please check the logs.")

    # Check for and Return the Plot: After the agent runs, check if the
    # plot file was created. If so, return it alongside the text answer.
    if os.path.exists(PLOT_FILENAME):
        return text_output, PLOT_FILENAME
    else:
        # If no plot was created, return None for the image output.
        return text_output, None


except Exception as e:
    
    error_message = f"An unexpected error occurred: {str(e)}"
    print(error_message) 
    return error_message, None

### 2.4 - Building the Gradio UI

Finally, we create the web interface. We use `gr.Blocks` for a custom layout. We define the input components (`gr.File`, `gr.Textbox`) and the output components (`gr.Markdown`, `gr.Image`).

The most important line is `submit_button.click(...)`. This tells Gradio that when the button is clicked, it should call our `data_analyst_agent` function, passing the content from the input components to it and sending the function's results to the output components.

In [ ]:
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue")) as demo:
    gr.Markdown(
        """
        # 🤖 Agentic Data Analysis with LangChain
        Upload your CSV file, ask a question in natural language, and the AI agent will work to find the answer.
        It can perform calculations, data manipulation, and even generate visualizations.
        """
    )
    
    with gr.Row():
        with gr.Column(scale=1):
            # Input components
            file_input = gr.File(label="Upload your CSV", file_types=[".csv"])
            text_input = gr.Textbox(
                label="What would you like to know?",
                placeholder="e.g., 'What is the correlation between column A and B?' or 'Create a bar chart of sales by category.'"
            )
            submit_button = gr.Button("🚀 Run Analysis", variant="primary")
        
        with gr.Column(scale=2):
            # Output components
            text_output = gr.Markdown(label="📝 Agent's Answer")
            plot_output = gr.Image(label="📊 Generated Visualization", type="filepath")

    # Connect the button to the agent function
    submit_button.click(
        fn=data_analyst_agent,
        inputs=[file_input, text_input],
        outputs=[text_output, plot_output]
    )
    
# To run this app, save it as app.py and run 'uv run app.py' in your terminal.
# if __name__ == "__main__":
#     demo.launch()

## Step 3: Running the Application

To run the app, use `uv` to execute the script from your terminal.

In [ ]:
!uv run app.py

error: Failed to spawn: `app.py`
  Caused by: program not found
